QUESTION

You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. Hyundai Heavy Industries is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

Description: Measurements of ship size, capacity, crew, and age for 158 cruise
ships.


Variables/Columns
Ship Name     1-20
Cruise Line   21-40
Age (as of 2013)   46-48
Tonnage (1000s of tons)   50-56
passengers (100s)   58-64
Length (100s of feet)  66-72
Cabins  (100s)   74-80
Passenger Density   82-88
Crew  (100s)   90-96

It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

In [72]:
import findspark
findspark.init('/home/anshul/spark-3.0.0-preview2-bin-hadoop2.7/')
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('cruise').getOrCreate()

In [73]:
df=spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

In [74]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [75]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



making cruise_line a categorical variable

In [76]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|          Azamara|    2|
|         Carnival|   22|
|        Celebrity|   10|
|            Costa|   11|
|          Crystal|    2|
|           Cunard|    3|
|           Disney|    2|
| Holland_American|   14|
|              MSC|    8|
|        Norwegian|   13|
|          Oceania|    3|
|           Orient|    1|
|              P&O|    6|
|         Princess|   17|
|Regent_Seven_Seas|    5|
|  Royal_Caribbean|   23|
|         Seabourn|    3|
|        Silversea|    4|
|             Star|    6|
|         Windstar|    3|
+-----------------+-----+



Indexing the Strings according to cruise_line counts

In [77]:
from pyspark.ml.feature import StringIndexer
indexer=StringIndexer(inputCol='Cruise_line',outputCol='Cruise_cat')
indexed=indexer.fit(df).transform(df)


In [78]:
indexed.select(['Cruise_line','Cruise_cat']).show()

+-----------+----------+
|Cruise_line|Cruise_cat|
+-----------+----------+
|    Azamara|      16.0|
|    Azamara|      16.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
+-----------+----------+
only showing top 20 rows



In [79]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_cat']

include a feature column in the dataset

In [80]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

combining the columns for a feature column


In [83]:
assembler=VectorAssembler(inputCols=['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density',
             'Cruise_cat'],
 outputCol='features')

transform the data

In [84]:
output=assembler.transform(indexed)

In [85]:
output.select(['crew','features']).show()


+----+--------------------+
|crew|            features|
+----+--------------------+
|3.55|[6.0,30.276999999...|
|3.55|[6.0,30.276999999...|
| 6.7|[26.0,47.262,14.8...|
|19.1|[11.0,110.0,29.74...|
|10.0|[17.0,101.353,26....|
| 9.2|[22.0,70.367,20.5...|
| 9.2|[15.0,70.367,20.5...|
| 9.2|[23.0,70.367,20.5...|
| 9.2|[19.0,70.367,20.5...|
|11.5|[6.0,110.23899999...|
|11.6|[10.0,110.0,29.74...|
| 6.6|[28.0,46.052,14.5...|
| 9.2|[18.0,70.367,20.5...|
| 9.2|[17.0,70.367,20.5...|
| 9.3|[11.0,86.0,21.24,...|
|11.6|[8.0,110.0,29.74,...|
|10.3|[9.0,88.5,21.24,9...|
| 9.2|[15.0,70.367,20.5...|
| 9.3|[12.0,88.5,21.24,...|
| 9.2|[20.0,70.367,20.5...|
+----+--------------------+
only showing top 20 rows



In [87]:
final_data=output.select(['features','crew'])

In [88]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



splitting the data

In [89]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [90]:
train_data.show()
test_data.show()

+--------------------+-----+
|            features| crew|
+--------------------+-----+
|[4.0,220.0,54.0,1...| 21.0|
|[5.0,115.0,35.74,...| 12.2|
|[5.0,122.0,28.5,1...|  6.7|
|[5.0,133.5,39.59,...|13.13|
|[5.0,160.0,36.34,...| 13.6|
|[6.0,30.276999999...| 3.55|
|[6.0,30.276999999...| 3.55|
|[6.0,112.0,38.0,9...| 10.9|
|[6.0,113.0,37.82,...| 12.0|
|[6.0,158.0,43.7,1...| 13.6|
|[7.0,89.6,25.5,9....| 9.87|
|[7.0,116.0,31.0,9...| 12.0|
|[7.0,158.0,43.7,1...| 13.6|
|[8.0,77.499,19.5,...|  9.0|
|[8.0,91.0,22.44,9...| 11.0|
|[9.0,81.0,21.44,9...| 10.0|
|[9.0,85.0,19.68,9...| 8.69|
|[9.0,90.09,25.01,...| 8.69|
|[9.0,105.0,27.2,8...|10.68|
|[9.0,110.0,29.74,...| 11.6|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
|            features| crew|
+--------------------+-----+
|[5.0,86.0,21.04,9...|  8.0|
|[6.0,90.0,20.0,9....|  9.0|
|[6.0,93.0,23.94,9...|11.09|
|[6.0,110.23899999...| 11.5|
|[8.0,110.0,29.74,...| 11.6|
|[9.0,59.058,17.0,...|  7.4|
|[9.0,88.5,21.24,

In [91]:
train_data.describe().show()
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              114|
|   mean|7.745350877192993|
| stddev|3.612161655117104|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                44|
|   mean| 7.920681818181819|
| stddev|3.2411463646171605|
|    min|              0.59|
|    max|              13.0|
+-------+------------------+



applying the linear regression

In [92]:
from pyspark.ml.regression import LinearRegression
lr=LinearRegression(labelCol='crew')

training the model

In [93]:
lr_model=lr.fit(train_data)

In [94]:
print("coefficeint:",lr_model.coefficients)
print("intercept:",lr_model.intercept)

coefficeint: [-0.01555702268214075,0.007079139583526512,-0.15188666072663484,0.3991276479779376,0.8842672872655041,-0.0019101453340354274,0.04146023672454371]
intercept: -0.8479635422854516


evaluate the model


In [95]:
result=lr_model.evaluate(test_data)

calculating the errors

In [96]:
print("RMSE:",result.rootMeanSquaredError)
print("MSE:",result.meanSquaredError)
print("R2:",result.r2)

RMSE: 0.9246416227553076
MSE: 0.8549621305315686
R2: 0.9167213238778045


In [97]:
from pyspark.sql.functions import corr

In [98]:
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [99]:
df.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

